In [1]:

from fpdf import FPDF
import pandas as pd
import datetime

# --- Configuration: UPDATED FILE NAME ---
FILE_PATH = 'data.xlsx'  # <--- **CHANGED TO YOUR ACTUAL FILE NAME**
OUTPUT_PDF_NAME = 'Automated_Report_Final.pdf'

# --- PDF Report Class Definition (Customizing Header and Footer) ---
class PDFReport(FPDF):
    def header(self):
        """Creates a professional, branded report header."""
        self.set_font('Arial', 'B', 15)
        self.set_fill_color(200, 220, 255) # Light Blue Background
        self.set_text_color(0, 51, 102) # Dark Blue Text
        self.cell(0, 10, 'Automated Sales Performance Report', 0, 1, 'C', 1)
        self.set_font('Arial', '', 10)
        self.set_text_color(0, 0, 0)
        self.cell(0, 5, f'Generation Date: {datetime.date.today().strftime("%Y-%m-%d")}', 0, 1, 'C')
        self.ln(5)

    def footer(self):
        """Creates a clean footer with page numbering."""
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.set_text_color(128, 128, 128) # Grey text
        self.cell(0, 10, f'Page {self.page_no()}/{{nb}}', 0, 0, 'C')

# --- Main Report Generation Function ---
def generate_report(file_path):
    # 1. READ DATA (Now focused on .xlsx)
    try:
        # Check file extension to determine the reader function
        if file_path.endswith(('.xlsx', '.xls')):
            # *** Use read_excel for .xlsx files ***
            df = pd.read_excel(file_path)
            print(f"Data successfully read from Excel file: {file_path}")
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
            print(f"Data successfully read from CSV file: {file_path}")
        else:
            raise ValueError("Unsupported file format. Please use .xlsx or .csv.")
    except FileNotFoundError:
        print(f"ERROR: File not found at {file_path}. Please check the file name and path.")
        return
    except Exception as e:
        print(f"ERROR reading file: {e}. If it's an Excel file, ensure 'openpyxl' is installed.")
        return

    # Standardize column names for analysis (ASSUME THESE ARE YOUR COLUMNS)
    # **IMPORTANT**: Adjust the column names below if they are different in your actual 'data.xlsx' file!
    try:
        df = df.rename(columns={
            'Total_Sale': 'Amount', 
            'Quantity': 'Units', 
            'Region': 'Category', 
            'Product': 'Item'
        }, errors='ignore')

        # Ensure core columns for analysis exist and are numeric where needed
        df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce').fillna(0)
        df['Units'] = pd.to_numeric(df['Units'], errors='coerce').fillna(0)
        
    except KeyError as e:
        print(f"ERROR: Required column not found or renamed unsuccessfully. Check your column headers: {e}")
        return
    
    # 2. ANALYZE DATA
    print("Analyzing data...")

    total_revenue = df['Amount'].sum()
    total_quantity = df['Units'].sum()
    
    # Top 3 Sales by Category
    category_sales = df.groupby('Category')['Amount'].sum().sort_values(ascending=False)
    top_3_categories = category_sales.head(3)

    # Best-Selling Item by Units
    item_performance = df.groupby('Item').agg({
        'Units': 'sum',
        'Amount': 'sum'
    }).sort_values(by='Units', ascending=False)
    best_item = item_performance.iloc[0]

    # Analysis Results Dictionary
    analysis_results = {
        "Total Revenue": f"${total_revenue:,.2f}",
        "Total Units Sold": f"{total_quantity:,}",
        "Top Selling Item": f"{best_item.name} (Units: {best_item['Units']:,}, Rev: ${best_item['Amount']:,.2f})",
        "Top 3 Categories": top_3_categories.to_dict()
    }
    
    # 3. GENERATE PDF REPORT
    
    pdf = PDFReport('P', 'mm', 'A4')
    pdf.alias_nb_pages()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    # --- Executive Summary Section ---
    pdf.set_font('Arial', 'B', 14)
    pdf.set_text_color(0, 51, 102) 
    pdf.cell(0, 10, 'Executive Summary', 0, 1, 'L')
    pdf.ln(2)

    pdf.set_font('Arial', '', 10)
    pdf.set_text_color(0, 0, 0) 

    # Key Metrics List
    for k, v in analysis_results.items():
        if k != "Top 3 Categories":
            pdf.write(5, f"  - {k}: ") 
            pdf.set_font('Arial', 'B', 10)
            pdf.write(5, str(v))
            pdf.set_font('Arial', '', 10)
            pdf.ln(5)
    
    # Top Categories Table
    pdf.ln(5)
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(0, 7, 'Category Sales Breakdown (Top 3):', 0, 1, 'L')
    
    # Table Header
    col_width = 50
    pdf.set_fill_color(240, 240, 240) 
    pdf.set_font('Arial', 'B', 10)
    pdf.cell(col_width, 7, 'Category', 1, 0, 'C', 1)
    pdf.cell(col_width, 7, 'Total Sales', 1, 1, 'C', 1)

    # Table Rows
    pdf.set_font('Arial', '', 10)
    for category, sales in top_3_categories.items():
        pdf.cell(col_width, 6, str(category), 1, 0, 'L')
        pdf.cell(col_width, 6, f"${sales:,.2f}", 1, 1, 'R')

    # --- Raw Data Sample Section ---
    pdf.ln(10)
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(0, 10, 'Data Sample (First 10 Rows)', 0, 1, 'L')

    # Data Table (Using first four columns for a generic view)
    data_sample = df.iloc[:, :4].head(10)
    col_headers = list(data_sample.columns)
    
    # Calculate column widths dynamically
    col_widths = [190 / len(col_headers) - 5] * len(col_headers) 
    
    # Data Sample Header
    pdf.set_fill_color(240, 240, 240)
    pdf.set_font('Arial', 'B', 8)
    for header, width in zip(col_headers, col_widths):
        pdf.cell(width, 7, header, 1, 0, 'C', 1)
    pdf.ln()

    # Data Sample Rows
    pdf.set_font('Arial', '', 8)
    for index, row in data_sample.iterrows():
        for item, width in zip(row.values, col_widths):
            if isinstance(item, (int, float)):
                 cell_value = f"{item:,.2f}" if width > 40 else str(int(item))
                 align = 'R'
            else:
                 cell_value = str(item)[:int(width / 2.5)]
                 align = 'L'
            
            pdf.cell(width, 6, cell_value, 1, 0, align)
        pdf.ln()
    
    # Output the PDF
    pdf.output(OUTPUT_PDF_NAME, 'F')
    print(f"\nSUCCESS: Report generated as '{OUTPUT_PDF_NAME}'")

# --- EXECUTION ---
if __name__ == '__main__':
    generate_report(FILE_PATH)

ERROR: File not found at data.xlsx. Please check the file name and path.
